In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail_3'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '30G'},
    default_reference='GRCh38'
) 

2023-04-16 15:02:50.172 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0079:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230416-1502-0.2.109-b71b065e4bb6.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

Loading BokehJS ...

#### define function for the oligogenic model:

In [6]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
def train_model(mt, table_name, geneset_name, top_genes_20): 
    
    geneset_lens = list(range(20))
    cadds = [0,5,10,16,20,25,30]
    
    mt = mt.explode_rows(mt.nearest_genes_20kb)
    mt = mt.filter_rows(
        hl.literal(top_genes_20).contains(mt.nearest_genes_20kb)
    )
    
    mt = mt.checkpoint(localfs_path+table_name+'top_genes.mt', overwrite = True)
    
    for c in cadds:
        
        mt = mt.group_rows_by(mt.nearest_genes_20kb).aggregate(
            n_non_ref_above_cadd = hl.agg.filter(
                mt.cadd.phred_score > c,
                hl.agg.sum(mt.GT.n_alt_alleles()
                          )
            ),
            weited_n_non_ref = hl.agg.filter(
                mt.cadd.phred_score > c,
                hl.agg.sum(mt.GTS.n_alt_alleles()*mt.cadd_phred_score)
            )
        )
        
        control_mean_per_gene = mt.aggregate_rows(
            hl.agg.filter(
                mt.group != 'GTS',
                hl.agg.mean(
                    mt.n_non_ref_above_cadd
                )
            )
        )
                                                 
        control_weited_mean_per_gene =  mt.aggregate_rows(
            hl.agg.filter(
                mt.group != 'GTS',
                hl.agg.mean(
                    mt.weited_n_non_ref
                )
            )
        )
        
        # i teraz odjąć trzeba to co dostałam od wyników dla próbek wszystkich i w zależności od tego co będzie to taki jest assigment i to dopiero collect, 
        # tylko w jakiś mądry zestaw żeby było wszystko na raz
        
        
        
        variants_gts[rows] = np.mean(np.array(non_refs)[categories])
                variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

                variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
                variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

        
        
                                        
    
    
    
    vars_controls = np.zeros((len(top_genes)))
    vars_gts = np.zeros((len(top_genes)))
    
    vars_controls = np.zeros((len(top_genes)))
    vars_gts = np.zeros((len(top_genes)))
    
    model_asignment = np.zeros((len(top_genes), 80))
    
    
    
    
    for s in sets:

        top_genes = skat.order_by('p_value').id.take(s)   

        for c in cadds:

            variants_controls = np.zeros((len(top_genes)))
            variants_gts = np.zeros((len(top_genes)))

            variants_controls_test = np.zeros((len(top_genes)))
            variants_gts_test = np.zeros((len(top_genes)))

            model_asignment = np.zeros((len(top_genes), 80))
            test_asignment = np.zeros((len(top_genes), 145)) 

            for rows, n in enumerate(top_genes):

                mt_skat_log = mt_for_skat.filter_rows(mt_for_skat.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
                mt_test_skat_log = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny

                mt_skat_log = mt_skat_log.filter_rows(mt_skat_log.cadd > c)
                mt_skat_log = mt_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
                non_refs = mt_skat_log.non_refs.collect()

                mt_test_skat_log = mt_test_skat_log.filter_rows(mt_test_skat_log.cadd > c)
                mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample, prepare also test matrix
                non_refs_test = mt_test_skat_log.non_refs.collect()

                variants_gts[rows] = np.mean(np.array(non_refs)[categories])
                variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

                variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
                variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

                results = (non_refs - variants_controls[rows]) 
                results_test = (non_refs_test - variants_controls[rows]) 

                model_asignment[rows] = (results)
                test_asignment[rows] = (results_test)

            model_asignment = np.sum(model_asignment, axis = 0)
            test_asignment = np.sum(test_asignment, axis = 0)

            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_gts'+str(s)+'cadd'+str(c), variants_gts)
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_controls'+str(s)+'cadd'+str(c), variants_controls)

            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_gts_test'+str(s)+'cadd'+str(c), variants_gts_test)
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_controls_test'+str(s)+'cadd'+str(c), variants_controls_test)

            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-model_asignment'+str(s)+'cadd'+str(c), model_asignment)
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-test_asignment'+str(s)+'cadd'+str(c), test_asignment)

            false_pos = []
            true_pos = []

            for x in np.linspace(-25,20,20000):
                false_pos.append(np.sum((test_asignment > x)[np.invert(categories_test)])/56)
                true_pos.append(np.sum((test_asignment > x)[categories_test])/89)

            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-false_pos'+str(s)+'cadd'+str(c), false_pos)
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-true_pos'+str(s)+'cadd'+str(c), true_pos)

            print(str(s)+'cadd'+str(c))
            print(variants_gts)
            print(variants_controls)

    false_pos = []
    true_pos = []

    for s in sets:
            for c in cadds:
                false_pos.append(np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-false_pos'+str(s)+'cadd'+str(c)+'.npy'))
                true_pos.append(np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-true_pos'+str(s)+'cadd'+str(c)+'.npy'))

    ROC

In [ ]:
def test_model(geneset_name, cadds, top_genes): 
    
    for c_idx, c in enumerate(cadds):

        test_asignment = np.zeros((len(top_genes), 145)) 
        mt_skat_log = hl.read_matrix_table(
            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt_for_skat_'+str(c)+'.mt'
        )
        mt_test_skat_log = hl.read_matrix_table(
            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt_test_'+str(c)+'.mt'
        )
      
        for rows, n in enumerate(top_genes):

            mt_skat_log_gene = mt_skat_log.filter_rows(mt_skat_log.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
        
            mt_test_skat_log_gene = mt_test_skat_log.filter_rows(mt_test_skat_log.nearest_genes_20kb.contains(n)) # to ma rodziny

            mt_skat_log_gene = mt_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log_gene.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
            non_refs = mt_skat_log_gene.non_refs.collect()
                                                
            mt_test_skat_log_gene = mt_test_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log_gene.GT.is_non_ref())) #count variants per sample, prepare also test matrix
            non_refs_test = mt_test_skat_log_gene.non_refs.collect()

            variants_controls = np.mean(np.array(non_refs)[np.invert(categories)])
                
            test_asignment[rows] = (non_refs_test - variants_controls)
            
            print(test_asignment.shape)
                
        for s in sets:
        
            test_asignment_subset = test_asignment[range(5-s,5), :]
            
            print(test_asignment_subset.shape)
                
            test_asignment_subset = np.sum(test_asignment_subset, axis = 0)
                
            false_pos = []
            true_pos = []
            
            for x in np.linspace(-100,100,10000):
            
                false_pos.append(np.sum((test_asignment_subset > x)[np.invert(categories_test)])/53)
                true_pos.append(np.sum((test_asignment_subset > x)[categories_test])/91)
            
            print(np.trapz(false_pos, true_pos))
        
            np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/false_pos'+str(s)+'cadd'+str(c)+'gene'+geneset_name, false_pos)
            np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/true_pos'+str(s)+'cadd'+str(c)+'gene'+geneset_name, true_pos)

def full_model(gene_list, geneset_name, mt_for_model):